# SageMaker HyperPod Custom Endpoint - Init Experience

This notebook demonstrates the complete end-to-end workflow for creating a SageMaker HyperPod Custom Endpoint using the HyperPod CLI. The init experience provides a guided approach to create Hyperpod Custom Endpoint with validation and configuration management.

## Prerequisites

- SageMaker HyperPod CLI installed (`pip install sagemaker-hyperpod`)
- Hyperpod custom inference template installed (`pip install hyperpod-custom-inference-template`)
- Hyperpod inference operator installed in your hyperpod cluster
- Python 3.8+ environment

## Workflow Overview

1. **Initialize** - Create initial custom endpoint configuration
2. **Configure** - Customize custom endpoint parameters
3. **Validate** - Verify configuration before deployment
4. **Create** - Deploy the custom endpoint creation
5. **Monitor** - Check custom endpoint status and manage lifecycle


## Step 0: Connect to your Hyperpod cluster

Make sure you have installed hyperpod inference operator in your hyperpod cluster.


In [ ]:
# List all available SageMaker HyperPod clusters in your account
!hyp list-cluster

In [ ]:
# Configure your local kubectl environment to interact with a specific SageMaker HyperPod cluster (and namespace)
!hyp set-cluster-context --cluster-name ml-cluster-integ-test

## Step 1: Initialize Custom Endpoint Configuration

The `hyp init hyp-custom-endpoint` command creates a new configuration template with default settings. This generates a `config.yaml` file that serves as the foundation for your deployment.

**What this does:**
- Creates a `config.yaml` with default custom endpoint settings.
- Creates a `k8s.jinja` which is a reference to the k8s payload that is going to be submitted with. Users can refer this to understand how the parameters are being used. 
- Creates a `README.md` which is a detailed explanation of the init experience.


In [ ]:
# Initialize a new custom endpoint configuration in the current directory
!hyp init hyp-custom-endpoint

## Step 2: Configure Custom Endpoint Settings

The `hyp configure` command allows you to customize your custom endpoint configuration.

**Key configuration options:**
- **model_name**: Name of model to create on SageMaker
- **instance_type**: EC2 instance type for the inference server
- **endpoint_name**: Name of SageMaker endpoint
- **model_source_type**: Source type: fsx or s3
- **image_uri**: Inference server image name
- **container_port**: Port on which the model server listens
- **model_volume_mount_name**: Path inside container for model volume

In [ ]:
!hyp configure --endpoint-name my-custom-endpoint

### View Current Configuration

Let's examine the generated configuration to understand what will be deployed:

In [ ]:
# Display the current configuration
!cat config.yaml | head -50

## Step 3: Validate Configuration

The `hyp validate` command performs syntax validation of your custom endpoint configuration before deployment. This helps catch configuration errors early and ensures all prerequisites are met.


In [ ]:
# Validate the custom endpoint configuration
# This checks for potential issues before deployment
!hyp validate

## Step 4: Reset Configuration (Optional)

The `hyp reset` command allows you to reset your configuration to defaults or clean up any partial deployments. This is useful when you want to start fresh or if validation reveals issues that require a clean slate.

**Use cases for reset:**
- Starting over with a clean configuration
- Cleaning up after failed deployments
- Switching between different custom endpoint configurations


In [ ]:
# Reset configuration if needed (uncomment to use)
# !hyp reset

print("Reset command available if configuration changes are needed")

## Step 5: Create the Custom Endpoint

The `hyp create` command deploys your HyperPod custom endpoint with configurations in the config.yaml. A timestamped folder is created in the `runs` folder, where the config.yaml and the values-injected k8s.yaml kubernates payload is saved.

**Note:** The sagemaker custom endpoint typically takes 10-15 minutes to be created.


In [ ]:
# Create the custom endpoint
!hyp create

## Step 6: Monitor Custom Endpoint Creation

While the custom endpoint is being created, you can monitor its progress using the describe and list commands. These provide real-time status updates on the deployment process.

In [ ]:
# Check custom endpoint creation status
import time

print("Monitoring custom endpoint progress...")
for i in range(5):
    print(f"\n--- Status Check {i+1} ---")
    !hyp describe hyp-custom-endpoint --name my-custom-endpoint
    time.sleep(30)  # Wait 30 seconds between checks

## Step 7: Invoke Sagemaker Endpoint

After the sagemaker endpoint is successfully created, you can use `hyp invoke hyp-custom-endpoint` command to do basic invocation of sagemaker endpoint.

In [ ]:
!hyp invoke hyp-custom-endpoint --endpoint-name my-custom-endpoint --body '{"inputs":"What is the capital of USA?"}'

## Step 8: Describe Custom Endpoint

The `hyp describe hyp-custom-endpoint` command provides detailed information about your custom endpoint deployment status and sagemaker endpoint status.

In [ ]:
# Get detailed information about the custom endpoint
!hyp describe hyp-custom-endpoint  --name my-custom-endpoint

## Step 9: List All Custom Endpoints

The `hyp list hyp-custom-endpoint` command shows all HyperPod custom endpoints in your account. This is useful for managing multiple custom endpoint deployments and getting an overview of your deployments.


In [ ]:
# List all custom endpoints in your account
!hyp list hyp-custom-endpoint

## Next Steps

After successfully creating your HyperPod Custom Endpoint, you can:

1. **Monitor Resources**: Check pod status with `hyp list-pods hyp-custom-endpoint`
2. **Access Logs**: View pod logs with `hyp get-logs hyp-custom-endpoint`


## Troubleshooting

If you encounter issues during Custom Endpoint creation:

- Use `hyp get-operator-logs hyp-custom-endpoint` to check potential operator log errors
- Verify AWS credentials and permissions
- Ensure resource quotas are sufficient
- Review the configuration file for syntax errors
- Use `hyp validate` to identify configuration issues

## Cleanup

To avoid ongoing charges, remember to delete your custom endpoint when no longer needed:

```bash
hyp delete hyp-custom-endpoint --name my-custom-endpoint
```
